# Project 2: LangChain RAG Project
## This Chatboat Have All Information But Only About
  * Artificial Intelligence  
  * Agentic AI
  * Means All About AI

# RAG-based QA System using LangChain, Pinecone, and Google Generative AI

This project notebook demonstrates the creation of a **Retrieval-Augmented Generation (RAG)**-based Question Answering (QA) system. It leverages powerful tools and APIs such as **LangChain**, **Pinecone**, and **Google Generative AI (Gemini Model)** to provide contextual answers to user queries based on uploaded documents. The notebook showcases a step-by-step process, from loading and splitting documents to creating embeddings, uploading data to Pinecone, and querying the system for responses.

## Key Features
1. **Pinecone for Vector Storage**  
   The project uses Pinecone as the vector database to store and retrieve document embeddings efficiently.  
   
2. **LangChain Integration**  
   LangChain simplifies document processing by handling text splitting, embedding generation, and connecting various AI components.  
   
3. **Google Generative AI (Gemini)**  
   Utilizes the Gemini 1.5 model for both embedding generation and language model-based reasoning to deliver intelligent and context-aware responses.  

4. **RAG Approach**  
   Combines retrieval and generation by searching for the most relevant document chunks and using an LLM to formulate coherent answers.  

5. **Flexible Querying**  
   Users can input any query, and the system retrieves the most relevant information from the indexed documents to provide accurate answers.  

## Workflow
1. **Setup**  
   Install and configure the required libraries such as LangChain, Pinecone Client, Google Generative AI, and TQDM.  

2. **Embedding Generation**  
   Process documents into chunks and generate embeddings using Google Generative AI's embedding model.  

3. **Vector Indexing**  
   Store the generated embeddings in a Pinecone index for efficient similarity-based retrieval.  

4. **Query Handling**  
   User queries are processed through a retrieval-augmented chain that fetches the most relevant documents, formulates a response, and displays the result.

5. **Source Document Traceability**  
   Returns the source documents used in formulating each response for transparency and validation.

## Use Cases
- Building document-based question-answering systems.
- Enhancing productivity by allowing quick access to relevant information from large datasets.
- Supporting tasks in education, research, and knowledge management.

## Technologies Used
- **LangChain**: Framework for building AI-powered applications.
- **Pinecone**: Vector database for storing and retrieving embeddings.
- **Google Generative AI (Gemini Model)**: Advanced generative AI for embedding and LLM capabilities.
- **Python**: Programming language for integrating APIs and data processing.

This project highlights the potential of combining vector search with generative AI to enable sophisticated knowledge retrieval systems.


In [1]:
!pip install -qU langchain langchain-pinecone tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
PINECONE_ENVIRONMENT = 'us-east-1'

In [3]:
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(
    api_key=PINECONE_API_KEY
)

# Check if the index exists; if not, create it
index_name = "anewindex"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",  # Choose the metric: cosine, euclidean, or dotproduct
        spec=ServerlessSpec(
            cloud="aws",
            region=PINECONE_ENVIRONMENT  # Use your environment's region
        )
    )

# # Connect to the index
index = pc.Index(name=index_name)
print(f"Successfully connected to index: {index_name}")


Successfully connected to index: anewindex


In [5]:
!pip install -q -U langchain-google-genai

In [6]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

# os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY # Assuming PINECONE_API_KEY is already defined


embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # Specify the desired embedding model
    api_key=GOOGLE_API_KEY
)

In [7]:
!pip install -q -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [8]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("/content/document.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [9]:

from tqdm import tqdm

# Create embeddings and upload to Pinecone
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)

    # Modify the upsert to use a dictionary for metadata
    index.upsert([{
        "id": doc.metadata["source"],  # Use "id" instead of the first tuple element
        "values": vector,  # Use "values" for the vector
        "metadata": {
            "text": doc.page_content,  # Add the text as part of metadata
            "source": doc.metadata["source"]  # Include the source
        }
    }])


100%|██████████| 21/21 [00:05<00:00,  3.69it/s]


In [10]:
from langchain_pinecone import PineconeVectorStore

# Use from_existing_index to load from an existing index
# The from_existing_index method in PineconeVectorStore takes the index_name and embedding directly
retriever = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough,RunnableMap

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

In [12]:
prompt = ChatPromptTemplate.from_messages([("human", message)])

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY,model="gemini-1.5-flash", temperature=0.7)

In [14]:
rag_chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"])[0], # Get the first relevant document
    "question": RunnablePassthrough()
}) | prompt | gemini_model


In [15]:
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA

# Create a vector store using the Pinecone index
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings
)

# Create the retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})  # Retrieve top 2 most similar documents

# Create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_model,
    chain_type="stuff",
    retriever=retriever,  # Pass the retriever here
    return_source_documents=True  # Optional: to get the source documents used in the response
)

In [18]:
vectorstore.similarity_search("Future?", k=3)

[Document(id='/content/skills.txt', metadata={'source': '/content/skills.txt'}, page_content='By consistently following these practices and observing these tech leaders, you’ll start building these skills. Since your goal is to create something valuable, the time invested now will create a foundation that will serve you well in your career.'),
 Document(id='/content/document.txt', metadata={'source': '/content/document.txt'}, page_content='By consistently following these practices and observing these tech leaders, you’ll start building these skills. Since your goal is to create something valuable, the time invested now will create a foundation that will serve you well in your career.'),
 Document(id='/content/stuff.txt', metadata={'source': '/content/stuff.txt'}, page_content='By consistently following these practices and observing these tech leaders, you’ll start building these skills. Since your goal is to create something valuable, the time invested now will create a foundation that

In [19]:
print("Welcome To RAG Project")
print('='*40)

query = "What is there in the file?"
print('Human Message:',query)
response = qa_chain.invoke(query)

# Print the answer
print("Agent Message:", response['result'])



Welcome To RAG Project
Human Message: What is inside the document?
Agent Message: The provided text is a short excerpt offering advice on skill-building by observing tech leaders and consistently following best practices.  It emphasizes the long-term value of this investment.  It doesn't describe the contents of a specific document.
